In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split
import math
import random
import scipy

# Load data from Excel file
# data = pd.read_excel('Temperature Dataset.xlsx', header=None).values
data = pd.read_excel('Video surveillance Dataset_V2.xlsx', header=None).values
num_data = data.shape[0]
# Normalize the input data
for ii in range(4): 
    data[:, ii] = (data[:, ii] / np.max(data[:, ii]) )
    
# Split the dataset into a training and testing set
# X_train, X_test,y_train, y_test = train_test_split(data[:num_data, :3], data[:num_data,3], test_size=0.25, random_state=42)

split_ratio_train = 0.7

split_line_number = int(np.shape(data)[0] * split_ratio_train)
X_train = data[:split_line_number, :3]
y_train = data[:split_line_number, 3]

other_data = data[split_line_number:, :4]

X_test = data[split_line_number:len(data), :3]
y_test = data[split_line_number:len(data), 3] 

def 𝑙𝑜𝑔𝑠𝑖𝑔(x):
    return 1/(1 + (scipy.special.expit(-1 * x)))

def 𝑡𝑎𝑛𝑠𝑖𝑔(x):
    return 2/(1+scipy.special.expit(-2*x))-1

def relu(x):
    return np.maximum(0, x)

def 𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒(x):
    return x

def sigmoid_derivative(x):
    return (1/(1 + scipy.special.expit(-x))) * (1 - 1/(1 + scipy.special.expit(-x)))
    
# def tanh_derivative(x):
#     return 1 - np.tanh(2 * x)**2

def tanh_derivative(x):
    tanh_x = tansig(x)
    return 1 - tanh_x**2

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

def pureline_derivative(x):
    return np.ones_like(x)


# 𝐹1(𝑥) = 𝛼 × 𝑙𝑜𝑔𝑠𝑖𝑔(𝑥) + 𝛽 × 𝑡𝑎𝑛𝑠𝑖𝑔(𝑥) + 𝛾 × 𝑅𝑒𝑙𝑢(𝑥) + 𝛿 × 𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒(𝑥)
def Function1(x,𝛼,𝛽,𝛾,𝛿): 
    return 𝛼.T * 𝑙𝑜𝑔𝑠𝑖𝑔(x) + 𝛽.T * 𝑡𝑎𝑛𝑠𝑖𝑔(x) + 𝛾.T * relu(x) + 𝛿.T * 𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒(x)

def Function1_derivative(x, 𝛼, 𝛽, 𝛾, 𝛿): 
    # Sum up the derivatives with the corresponding coefficients
    result = 𝛼* sigmoid_derivative(x).T  + 𝛽 * tanh_derivative(x).T + 𝛾 * relu_derivative(x).T + 𝛿 * pureline_derivative(x).T
    return np.diag(result.flatten())


def Function2(x,𝛼,𝛽,𝛾,𝛿):   
    return  𝛼 * 𝑙𝑜𝑔𝑠𝑖𝑔(x) + 𝛽 * 𝑡𝑎𝑛𝑠𝑖𝑔(x) + 𝛾 * relu(x) 

def Function2_derivative(x, 𝛼, 𝛽, 𝛾, 𝛿): 
    # Sum up the derivatives with the corresponding coefficients
    result = 𝛼 * sigmoid_derivative(x) + 𝛽 * tanh_derivative(x) + 𝛾 * relu_derivative(x)
    return np.diag((np.array(result).flatten()))

# Define the number of input, hidden, and output neurons
input_neurons = X_train.shape[1]
l1_neurons = 3
l2_neurons = 30
output_neurons = 1  # Linear activation for regression

# Initialize the weights with random values in range (-1,1)
np.random.seed(1)
w1 = 2 * np.random.random((input_neurons, l1_neurons)) - 1
w2 = 2 * np.random.random((l1_neurons, l2_neurons)) - 1
w3 = 2 * np.random.random((l2_neurons, output_neurons)) - 1
# print('w1: ', w1, 'w2: ',w2, 'w3: ',w3)


𝛼,𝛽,𝛾,𝛿 = round(random.uniform(-1, 1), 2),round(random.uniform(-1, 1), 2),round(random.uniform(-1, 1), 2),round(random.uniform(-1, 1), 2)

𝛼1,𝛽1,𝛾1,𝛿1 =  np.random.uniform(-1, 1, (l1_neurons, 1)) ,np.random.uniform(-1, 1, (l1_neurons, 1)),np.random.uniform(-1, 1, (l1_neurons, 1)),np.random.uniform(-1, 1, (l1_neurons, 1))
𝛼2,𝛽2,𝛾2,𝛿2 = np.random.uniform(-1, 1, (l2_neurons, 1)),np.random.uniform(-1, 1, (l2_neurons, 1)),np.random.uniform(-1, 1, (l2_neurons, 1)),np.random.uniform(-1, 1, (l2_neurons, 1))
𝛼3,𝛽3,𝛾3,𝛿3 = np.random.uniform(-1, 1, (output_neurons, 1)),np.random.uniform(-1, 1, (output_neurons, 1)),np.random.uniform(-1, 1, (output_neurons, 1)),np.random.uniform(-1, 1, (output_neurons, 1))

# Training parameters
learning_rate = 0.02
epochs = 40  # Train sample by sample

mse_train = np.zeros(epochs)
mse_test = np.zeros(epochs)

In [ ]:
# Training the MLP for regression
for epoch in range(epochs):
    print('epoch: ', epoch)
    # Shuffle the training data in each epoch
#     shuffle_indices = np.arange(len(X_train))
#     np.random.shuffle(shuffle_indices)
#     X_train = X_train[shuffle_indices]
#     y_train = y_train[shuffle_indices]


    total_error = 0
    error_data_train = np.zeros(len(X_train))
    output_data_train = np.zeros(len(X_train))
    for i in range(len(X_train)):
        #-------------------------------- Feed Forward -------------------------------------
        input_layer = X_train[i:i+1]

        net1 = np.dot(input_layer, w1)   # net1 = x * w1
        o1   = Function1(net1,𝛼1,𝛽1,𝛾1,𝛿1) #  o1 = f(net1)
        net2 = np.dot(o1, w2) # net2 = o1 * w2
        o2   = Function1(net2,𝛼2,𝛽2,𝛾2,𝛿2) #  o2 = f(net2)
        net3 = np.dot(o2, w3)   # net3 = o2 * w3
        o3   = Function1(net3,𝛼3,𝛽3,𝛾3,𝛿3) #  o3 = f(net3)
        
        output_data_train[i] = o3
        
        #-------------------------------- Backpropagation ----------------------------------- 
        output_layer_error = y_train[i:i+1] - output_data_train[i]
        

        # dE/da = dE/de * de/do3 * do3/da => o3=F(net3,a)= a*logsig + ... => dE/da = e * -1 * logsig(net3)
        diag_𝑙𝑜𝑔𝑠𝑖𝑔_net3 = np.array(np.diag(𝑙𝑜𝑔𝑠𝑖𝑔(net3.flatten())))
        diag_𝑡𝑎𝑛𝑠𝑖𝑔_net3 = np.array(np.diag(𝑡𝑎𝑛𝑠𝑖𝑔(net3.flatten())))
        diag_relu_derivative_net3 = np.array(np.diag(relu_derivative(net3.flatten()))) 
        diag_𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_net3 = np.array(np.diag(𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒(net3.flatten())))
        
        𝛼3 = 𝛼3 + learning_rate * output_layer_error * diag_𝑙𝑜𝑔𝑠𝑖𝑔_net3
        𝛽3 = 𝛽3 + learning_rate * output_layer_error * diag_𝑡𝑎𝑛𝑠𝑖𝑔_net3
        𝛾3 = 𝛾3 + learning_rate * output_layer_error * diag_relu_derivative_net3
        𝛿3 = 𝛿3 + learning_rate * output_layer_error * diag_𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_net3
        
        # update w3
        w3_old = w3
        fprim_net3 = np.array(Function1_derivative(net3,𝛼3,𝛽3,𝛾3,𝛿3)).reshape(-1,1) 
        w3 = w3 +  learning_rate * output_layer_error * o2.T.dot(fprim_net3) 
        
        diag_𝑙𝑜𝑔𝑠𝑖𝑔_net2 = np.array(np.diag(𝑙𝑜𝑔𝑠𝑖𝑔(net2.flatten()))) 
        diag_𝑡𝑎𝑛𝑠𝑖𝑔_net2= np.array(np.diag(𝑡𝑎𝑛𝑠𝑖𝑔(net2.flatten())))  
        diag_relu_derivative_net2 = np.array(np.diag(relu_derivative(net2.flatten()))) 
        diag_𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_net2 = np.array(np.diag(𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒(net2.flatten())))  
        
        # dE/da = dE/de * de/do3 * do3/dnet3 * dnet3/do2 * do2/da2 
        # => o2=F(net2,a)= a*logsig + ... => dE/da = e * -1 *F'(net3) * w3_old * logsig(net2)
        𝛼2 = 𝛼2 + learning_rate * output_layer_error * diag_𝑙𝑜𝑔𝑠𝑖𝑔_net2 @ w3_old @ fprim_net3 
        𝛽2 = 𝛽2 + learning_rate * output_layer_error * diag_𝑡𝑎𝑛𝑠𝑖𝑔_net2 @  w3_old @ fprim_net3 
        𝛾2 = 𝛾2 + learning_rate * output_layer_error *  diag_relu_derivative_net2 @ w3_old @ fprim_net3 
        𝛿2 = 𝛿2 + learning_rate * output_layer_error * diag_𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_net2 @ w3_old @ fprim_net3 
    
        # update w2
        w2_old = w2
        # Create a diagonal matrix from the 1D array
        fprim_net2 = np.array(Function1_derivative(net2,𝛼2,𝛽2,𝛾2,𝛿2))
#       w2 = w2 + learning_rate * output_layer_error * w3_old.dot(np.dot(np.dot(o1.T,fprim_net2),fprim_net3))
        w2 = w2 + learning_rate * output_layer_error * np.dot(np.dot(o1.T,fprim_net3),np.dot(w3.T,fprim_net2))

    
        # dE/da = dE/de * de/do3 * do3/dnet3 * dnet3/do2 * do2/dnet2 *dnet2/do1 * do1/da1
        # => o1=F(net1,a)= a*logsig + ... => dE/da = e * -1 *F'(net3) * w3 * F'(net2) *w2 * logsig(net1)
        diag_𝑙𝑜𝑔𝑠𝑖𝑔_net1 = np.array(np.diag(𝑙𝑜𝑔𝑠𝑖𝑔(net1.flatten())))
        diag_𝑡𝑎𝑛𝑠𝑖𝑔_net1= np.array(np.diag(𝑡𝑎𝑛𝑠𝑖𝑔(net1.flatten())))
        diag_relu_derivative_net1 = np.array(np.diag(relu_derivative(net1.flatten())))
        diag_𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_net1= np.array(np.diag(𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒(net1.flatten())))
        
        𝛼1 = 𝛼1 + learning_rate * output_layer_error * diag_𝑙𝑜𝑔𝑠𝑖𝑔_net1 @ w2_old @ fprim_net2 @ w3_old @ fprim_net3  
        𝛽1 = 𝛽1 + learning_rate * output_layer_error * diag_𝑡𝑎𝑛𝑠𝑖𝑔_net1 @ w2_old @ fprim_net2 @ w3_old @ fprim_net3 
        𝛾1 = 𝛾1 + learning_rate * output_layer_error * diag_relu_derivative_net1 @ w2_old @ fprim_net2 @ w3_old @ fprim_net3 
        𝛿1 = 𝛿1 + learning_rate * output_layer_error *  diag_𝑃𝑢𝑟𝑒𝑙𝑖𝑛𝑒_net1 @ w2_old @ fprim_net2 @ w3_old @ fprim_net3 
        
        # update w1        
        fprim_net1 = np.array(Function1_derivative(net1,𝛼1,𝛽1,𝛾1,𝛿1))
        w1 = w1 + learning_rate * output_layer_error * input_layer.T.dot(np.dot(fprim_net3 ,np.dot(np.dot(w3_old.T,fprim_net2),np.dot(w2_old.T,fprim_net1))))
    

        error_data_train[i] = output_layer_error
        total_error += np.abs(output_layer_error)
    
    mse_train[epoch] = np.mean(error_data_train ** 2)
    
    # Plotting the training data and output
    plt.figure(figsize=(20, 8))
    plt.subplot(2, 2, 1)
    plt.plot(y_train)
    plt.plot(output_data_train, 'r', linewidth=0.5)
    plt.xlabel('Train Data')
    plt.ylabel('Output')
    plt.legend(['Actual', 'Predicted'])

    # Plotting the training MSE
    plt.subplot(2, 2, 2)
    plt.semilogy(np.arange(1, epoch + 1), mse_train[:epoch])
    plt.xlabel('Epoch')
    plt.ylabel('MSE Train') 
    
    print('Epoch: {} \t'.format(epoch+1))
    print('total_error: ',total_error)
    print('MSE_train: {:.4f}'.format(mse_train[epoch]))
    
    plt.tight_layout()
    plt.show()
      
    print("\n\033[1;m" + "*" * 125)
        

#     if epoch % 100 == 0:
#         print(f"Epoch {epoch}: Total Error = {total_error[epoch]}")

print("****************************** Training completed *******************************")


In [ ]:
# Testing the trained MLP for regression
total_accuracy = 0 
error_data_test = np.zeros(len(X_test))
output_data_test = np.zeros(len(X_test))
for i in range(len(X_test)):
    input_layer = X_test[i:i+1]

    net1 = np.dot(input_layer, w1)   # net1 = x * w1
    o1   = Function1(net1,𝛼1,𝛽1,𝛾1,𝛿1) #  o1 = f(net1)
      
    net2 = np.dot(o1, w2) # net2 = o1 * w2
    o2   = Function1(net2,𝛼2,𝛽2,𝛾2,𝛿2) #  o2 = f(net2)
        
    net3 = np.dot(o2, w3)   # net3 = o2 * w3
    o3   = Function1(net3,𝛼3,𝛽3,𝛾3,𝛿3) #  o3 = f(net3)
    
    output_data_test[i] = o3
    error_data_test[i] = y_test[i] -output_data_test[i]


# Plotting the test data and output
plt.figure(figsize=(20, 8))
plt.subplot(2, 2, 1)
plt.plot(y_test)
plt.plot(output_data_test, 'r', linewidth=0.5)
plt.xlabel('Test Data')
plt.ylabel('Output')
plt.legend(['Actual', 'Predicted'])

# Plotting the training MSE
mse_train[epoch] = np.mean(error_data_train ** 2)
plt.subplot(2, 2, 2)
plt.semilogy(np.arange(1, epoch + 1), mse_train[:epoch])
plt.xlabel('Epoch')
plt.ylabel('MSE Train') 
    
print('Epoch: {} \t'.format(epoch+1))
print('total_error: ',total_error)
print('MSE_train: {:.4f}'.format(mse_train[epoch]))



# Plotting the test MSE
mse_test[epoch] = np.mean(error_data_test ** 2)
plt.subplot(2, 2, 2)
plt.semilogy(np.arange(1, epoch + 1), mse_test[:epoch])
plt.xlabel('Epoch')
plt.ylabel('MSE Test') 
    
print('Epoch: {} \t'.format(epoch+1))
print('MSE_test: {:.4f}'.format(mse_test[epoch]))
    

plt.figure(2)
m_train , b_train = np.polyfit(y_train, output_data_train, 1)    
plt.scatter(y_train, output_data_train,facecolors='none',edgecolors='#104E8B')
plt.plot(y_train, m_train*y_train+b_train,'r') 
plt.title('Regression Train') 

plt.figure(3)
m_test , b_test = np.polyfit(y_test, output_data_test, 1)  
plt.scatter(y_test, output_data_test,facecolors='none',edgecolors='#104E8B')
plt.plot(y_test, m_test*y_test+b_test,'r')
plt.title('Regression Test')
 
plt.tight_layout()
plt.show()

mse_train_result = mse_train[-1]
mse_test_result = mse_test[-1]

print("Final MSE on Train Data:", mse_train_result)
print("Final MSE on Test Data:", mse_test_result)
